# Using the `waterdata` module to pull data from the USGS Water Data APIs
The `waterdata` module will eventually replace the `nwis` module for accessing USGS water data. It leverages the [Water Data APIs](https://api.waterdata.usgs.gov/) to download metadata, daily values, and instantaneous values. 

While the specifics of this transition timeline are hazy, it is advised to switch to the new functions as soon as possible to reduce unexpected interruptions in your workflow.

As always, please report any issues you encounter on our [Issues](https://github.com/DOI-USGS/dataretrieval-python/issues) page. If you have questions or need help, please reach out to us at comptools@usgs.gov.

## Prerequisite: Get your Water Data API key
We highly suggest signing up for your own API key [here](https://api.waterdata.usgs.gov/signup/) to afford yourself higher rate limits and more reliable access to the data. If you opt not to register for an API key, then the number of requests you can make to the Water Data APIs is considerably lower, and if you share an IP address across users or workflows, you may hit those limits even faster. Luckily, registering for an API key is free and easy.

Once you've copied your API key and saved it in a safe place, you can set it as an environment variable in your python script for the current session:

```python
import os
os.environ['API_USGS_PAT'] = 'your_api_key_here'
``` 
Note that the environment variable name is `API_USGS_PAT`, which stands for "API USGS Personal Access Token".

If you'd like a more permanent repository-specific solution, you can use the `python-dotenv` package to read your API key from a `.env` file in your repository root directory, like this:

```python
!pip install python-dotenv # only run this line once to install the package in your environment
from dotenv import load_dotenv
load_dotenv()  # this will load the environment variables from the .env file
```
Make sure your `.env` file contains the following line:
```
API_USGS_PAT=your_api_key_here
```
Also, do not commit your `.env` file to version control, as it contains sensitive information. You can add it to your `.gitignore` file to prevent accidental commits.

## Lay of the Land
Now that your API key is configured, it's time to take a 10,000-ft view of the functions in the `waterdata` module.

### Metadata endpoints
These functions retrieve metadata tables that can be used to refine your data requests.

- `get_reference_table()` - Not sure which parameter code you're looking for, or which hydrologic unit your study area is in? This function will help you find the right input values for the data endpoints to retrieve the information you want.
- `get_codes()` - Similar to `get_reference_table()`, this function retrieves dataframes containing available input values that correspond to the Samples database.

### Data endpoints
- `get_daily()` - Daily values for monitoring locations, parameters, stat codes, and more.
- `get_continuous()` - Instantaneous values for monitoring locations, parameters, statistical codes, and more.
- `get_monitoring_locations()`- Monitoring location information such as name, monitoring location ID, latitude, longitude, huc code, site types, and more.
- `get_time_series_metadata()` - Timeseries metadata across monitoring locations, parameter codes, statistical codes, and more. Can be used to answer the question: what types of data are collected at my site(s) of interest and over what time period are/were they collected? 
- `get_latest_continuous()` - Latest instantaneous values for requested monitoring locations, parameter codes, statistical codes, and more.
- `get_latest_daily()` - Latest daily values for requested monitoring locations, parameter codes, statistical codes, and more.
- `get_field_measurements()` - Physically measured values (a.k.a discrete) of gage height, discharge, groundwater levels, and more for requested monitoring locations.
- `get_samples()` - Discrete water quality sample results for monitoring locations, observed properties, and more.

## Examples
Let's get into some examples using the functions listed above. First, we need to load the `waterdata` module.

In [ ]:
from IPython.display import display
from dataretrieval import waterdata

In [ ]:
pcodes,metadata = waterdata.get_reference_table("parameter-codes")
display(pcodes.head())

What is this `metadata` element? Let's take a look:

In [ ]:
metadata

All of these functions return Tuples containing a dataframe and a metadata element containing descriptors about the request made. This `BaseMetadata` object contains the request URL.

Let's say we want to find all parameter codes relating to streamflow discharge. We can use some string matching to find applicable codes.

In [ ]:
streamflow_pcodes = pcodes[pcodes['parameter_name'].str.contains('streamflow|discharge', case=False, na=False)]
display(streamflow_pcodes[['parameter_code_id', 'parameter_name']])

Interesting that there are so many different streamflow-related parameter codes! Going on experience, let's use the most common one, `00060`, which is "Discharge, cubic feet per second".

Now that we know which parameter code we want to use, let's find all the stream monitoring locations that have recent discharge data and at least 10 years of daily values in the state of Nebraska. 

In [ ]:
NE_locations,_ = waterdata.get_monitoring_locations(state_name="Nebraska", site_type_code="ST")
display(NE_locations.head())